In [ ]:
# https://iamtrask.github.io/2015/07/12/basic-python-network/
X = np.array([[0,0,1], # 0
              [0,1,1], # 1
              [1,0,1], # 1
              [1,1,1]])# 0

y = np.array([[0,1,1,0]]).T
syn0 = 2*np.random.random((3,4)) - 1  # (3,4)
syn1 = 2*np.random.random((4,1)) - 1  #(4,1)
for j in range(6000):
    l1 = 1/(1+np.exp(-(np.dot(X,syn0)))) #(4,4)
    l2 = 1/(1+np.exp(-(np.dot(l1,syn1)))) #(4,1)
    l2_delta = (y - l2)*(l2*(1-l2))
    l1_delta = l2_delta.dot(syn1.T) * (l1 * (1-l1))
    syn1 += l1.T.dot(l2_delta)
    syn0 += X.T.dot(l1_delta)








import numpy as np; np.set_printoptions(suppress=True)

def sig(x):
  return 1/(1 + np.exp(-x))

A = np.array([[0,0,1],
              [0,1,1],
              [1,0,1],
              [1,1,1]])

target = [[0],
          [1],
          [1],
          [0]]

L1 = np.array([[-10, -5, -5,   5],
               [-10,  1, -5,   5],
               [  5, -1, 10, -10]])

L2 = np.array([[-1],[1],[1],[-1]])

H1 = sig(np.dot(A,L1))

#array([[0.99330715, 0.26894142, 0.9999546 , 0.0000454 ],    -1    -.99+.26+.99+0=.26  
#       [0.00669285, 0.5       , 0.99330715, 0.00669285],     1     0+.5+1+0 = 1.5
#       [0.00669285, 0.00247262, 0.99330715, 0.00669285],     1      1
#       [0.00000031, 0.00669285, 0.5       , 0.5       ]])   -1    0

# Note how L2 is used to manage H2 so that the values come out easy


# np.dot(A[0,:],L1[:,0]) 
H2 = sig(np.dot(H1,L2))




In [ ]:
import torch 
x, y = torch.tensor(1.),  torch.tensor(2.)
w = torch.tensor(1., requires_grad=True)
for _ in range(200):
  loss = (((w*x)-y)**2)
  loss.backward()
  with torch.no_grad():
    w-= .01 * w.grad